In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !unzip "/content/drive/MyDrive/Colab Notebooks/News Data/archive (3).zip" -d "/content/drive/MyDrive/Colab Notebooks/News Data"

In [14]:
import numpy as np
import spacy
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [15]:
fake_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/News Data/Fake.csv")
true_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/News Data/True.csv")

In [16]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [17]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [18]:
true_df.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [19]:
fake_df.isnull().sum()

,0
title,0
text,0
subject,0
date,0


In [20]:
true_df = true_df.drop('date' ,axis = 1)
fake_df = fake_df.drop('date' ,axis = 1)

In [21]:
true_df['text'] = true_df['text'].str.lower()
true_df['title'] = true_df['title'].str.lower()
true_df['subject'] = true_df['subject'].str.lower()
fake_df['text'] = fake_df['text'].str.lower()
fake_df['title'] = fake_df['title'].str.lower()
fake_df['subject'] = fake_df['subject'].str.lower()

In [22]:
# Remove punctuation and numbers from title and text
fake_df['title'] = fake_df['title'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
fake_df['text'] = fake_df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
true_df['title'] = true_df['title'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
true_df['text'] = true_df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

In [23]:
# using spaCy English model
nlp = spacy.load("en_core_web_sm")

def clean_title(text):
    doc = nlp(text)
    cleaned_tokens = [
        token.lemma_.lower()  # lemmatize and lowercase
        for token in doc
        if not token.is_stop and not token.is_punct and not token.like_num
    ]
    return " ".join(cleaned_tokens)

# Apply the cleaning function on the 'title' and 'text' column
true_df['cleaned_title'] = true_df['title'].apply(clean_title)
true_df['cleaned_text'] = true_df['text'].apply(clean_title)

In [24]:
fake_df['cleaned_title'] = fake_df['title'].apply(clean_title)
fake_df['cleaned_text'] = fake_df['text'].apply(clean_title)

In [25]:
true_df['label'] = 0
fake_df['label'] = 1

In [26]:
fake_df['combined_text'] = fake_df['cleaned_title'] + " " + fake_df['cleaned_text']
true_df['combined_text'] = true_df['cleaned_title'] + " " + true_df['cleaned_text']
df = pd.concat([true_df, fake_df], ignore_index=True) # Combine them
df = df.sample(frac=1).reset_index(drop=True) # Shuffle
print(df['label'].value_counts())

label
1    23481
0    21417
Name: count, dtype: int64


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['combined_text'])  # or just 'clean_title'


In [28]:
from sklearn.model_selection import train_test_split
y = df['label']  # or whatever the column is named

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [29]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [30]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.987305122494432
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      4332
           1       0.99      0.99      0.99      4648

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

